### Project Fletcher Airline Sentiment Analysis

In [1]:
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import pylab
%matplotlib inline

from nltk.corpus import twitter_samples
from nltk.tokenize.casual import TweetTokenizer

from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score, confusion_matrix

##### Define Preprocessing and Tokenizer Functions to Read into Vectorizer

In [ ]:
def preprocess(text):
    text = re.sub(r"(?:\https?\://)\S+", "", text) #remove urls
    text = re.sub('\@(\w+)', " ", text).replace(": ","") #remove usernames
    text = re.sub('#(\w+)', " ", text) #remove hashtags
    text = text.replace("RT ","") #remove RT Symbols
    text = text.replace("RT: ","") #remove RT Symbols
    text = re.sub("[^a-zA-Z,]+", " ", text) #remove other non-alpha characters
    text = text.strip(" ") #remove leading and trailing whitespace
    
    return text

def tokenize(text):
    tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)  
    
    return [x for x in tokenizer.tokenize(preprocess(text)) if len(x)>=4]

#### Read-In NLTK Twitter Sentiment Data

In [ ]:
def read_demo_tweets():
    tweets = []
    for tweet in twitter_samples.strings("negative_tweets.json"):
        tweets.append({"text":tweet, "sentiment":1})
        
    for tweet in twitter_samples.strings("positive_tweets.json"):
        tweets.append({"text":tweet, "sentiment":0})
        
    return pd.DataFrame(tweets)


demo_tweets=read_demo_tweets()
print demo_tweets.sentiment.value_counts(dropna=False)
demo_tweets.head(5)

#### Read in CrowdFlower Data. Split Unambigous Tweets into Train and Test

In [ ]:
cf = pd.read_pickle("data/crowdflower/airline_sentiment_with_tb_vader.pkl")
cf.head(5)

In [ ]:
print cf.text_blob_sentiment.describe()
print cf.vader_sentiment.describe()

In [ ]:
cf_unambig = cf.ix[(cf["_trusted_judgments"]>2) & (cf["airline_sentiment:confidence"]==1.00)]
cf_ambig = cf.ix[(cf["_trusted_judgments"]>2) & (cf["airline_sentiment:confidence"]<1.00)]

cf_train, cf_test = train_test_split(cf_unambig, test_size=0.3, random_state=4444)

print cf_ambig.shape, cf_unambig.shape, cf_train.shape, cf_test.shape

#### Graph Positive/ Neutral vs. Negative Ratings by Airline

In [ ]:
negative = cf_unambig.ix[cf_unambig.sentiment==1].groupby(by="airline", as_index=False)["_unit_id"].count()
positive = cf_unambig.ix[cf_unambig.sentiment==0].groupby(by="airline", as_index=False)["_unit_id"].count()

negative.rename(columns={"_unit_id":"negative_tweets"}, inplace=True)
positive.rename(columns={"_unit_id":"positive_tweets"}, inplace=True)
total = pd.merge(negative, positive, on="airline")
total["total_tweets"] = total.negative_tweets + total.positive_tweets
total.sort_values(by="total_tweets", ascending=True, inplace=True)
print total.airline.tolist()

blue = sns.color_palette("deep")[0]
red = sns.color_palette("deep")[2]

plt.figure(figsize=(20,10))
p1=plt.barh(np.arange(6), total.negative_tweets, 0.8, color=red, label="Negative")
p2=plt.barh(np.arange(6), total.positive_tweets, 0.8, color=blue, left=total.negative_tweets,label="Positive/ Neutral")

#plt.title("Tweets by Sentiment and Airline", fontsize=32)
plt.xlabel("Number of Tweets (Feb 2015)", fontsize=26)
#plt.yticks([(x+(0.75/2)) for x in range(0,6)],[])
plt.tick_params(axis="x", labelsize=24)
plt.tick_params(axis="y", colors="white")
plt.legend(loc="best", fontsize=26)
plt.savefig("graphics/twees_by_sentiment_airline.png")

#### Grid Search to Optimize Lexicon-Based Sentiment Analyzers

In [ ]:
def lexicon_grid_search(variable, test, step):
    max_accuracy = 0
    max_param = np.nan
    
    for x in np.arange(-1, 1, step):
        pred_y = []
        
        for each in test[variable]:
            if each < x:
                pred_y.append(1) #negative
            else:
                pred_y.append(0) #positive/ neutral
            
        true_y = test["sentiment"]
        accuracy = accuracy_score(true_y, pred_y)
            
        if accuracy > max_accuracy:
            max_accuracy = accuracy
            max_param = x
            
    return (max_param, max_accuracy)

print "Text Blob 0.5: "+str(lexicon_grid_search("text_blob_sentiment", cf_test, 0.5))
print "Text Blob 0.1: "+str(lexicon_grid_search("text_blob_sentiment", cf_test, 0.1))
print "Text Blob 0.05: "+str(lexicon_grid_search("text_blob_sentiment", cf_test, 0.05))
print ""
print "VADER 0.5: "+str(lexicon_grid_search("vader_sentiment", cf_test, 0.5))
print "VADER 0.1: "+str(lexicon_grid_search("vader_sentiment", cf_test, 0.1))
print "VADER 0.05: "+str(lexicon_grid_search("vader_sentiment", cf_test, 0.05))

The optimal TextBlob Sentiment Classifer attains a 74% accuracy on the test set

#### Test Naive Bayes and SVM Models Trained on NLTK and Crowdflower Corpuses

#### Define Functions

In [ ]:
air_travel_sws = ENGLISH_STOP_WORDS.union(["aa","united","usairways","americanair","southwestair","jetblue","http",\
                                           "virginamerica","amp","flight","flights","plane","gate","flightled",\
                                           "bag","airline","airport","fly"])

In [ ]:
def fit_count_model(classifier, train, test, sws, binary):
    vectorizer = CountVectorizer(decode_error="ignore", ngram_range=(1,1), min_df=2, max_df=0.2, stop_words=sws,\
                                 tokenizer=tokenize, binary=binary)
    
    vectorizer.fit(train["text"])
    
    train_X = vectorizer.transform(train["text"])
    test_X = vectorizer.transform(test["text"])
    
    train_y = train["sentiment"]
    test_y = test["sentiment"]
    
    clf = classifier.fit(train_X, train_y)
    
    #print "Confusion Matrix: "
    #print confusion_matrix(test_y, clf.predict(test_X))
    
    accuracy = accuracy_score(test_y, clf.predict(test_X))
    features = zip(vectorizer.get_feature_names(), clf.coef_[0])
    top10_features = sorted(features, reverse=True, key=lambda x: x[1])[:10]
    
    print "Accuracy: "+str(accuracy)
    
    return [accuracy, [x[0] for x in top10_features]]
    

def fit_tfidf_model(classifier, train, test, sws, binary):
    vectorizer = TfidfVectorizer(decode_error="ignore", ngram_range=(1,1), min_df=2, max_df=0.2, stop_words=sws,
                                 tokenizer=tokenize, binary=binary)
        
    vectorizer.fit(train["text"])
    
    train_X = vectorizer.transform(train["text"])
    test_X = vectorizer.transform(test["text"])
    
    train_y = train["sentiment"]
    test_y = test["sentiment"]
    
    clf = classifier.fit(train_X, train_y)
    
    #print "Confusion Matrix: "
    #print confusion_matrix(test_y, clf.predict(test_X))
    
    accuracy = accuracy_score(test_y, clf.predict(test_X))
    features = zip(vectorizer.get_feature_names(), clf.coef_[0])
    top10_features=sorted(features, reverse=True, key=lambda x: x[1])[:10]
    
    print "Accuracy: "+str(accuracy)
    
    return [accuracy, [x[0] for x in top10_features]]


def test_models(function, train, sws, label):
    m1=function(BernoulliNB(binarize=None), train, cf_test, sws, True)
    m2=function(MultinomialNB(), train, cf_test, sws, False)
    m3=function(LinearSVC(), train, cf_test, sws, False)
    
    data = [{"clf":"Bernoulli NB", "accuracy":m1[0]*100, "features":m1[1], "label":label},\
            {"clf":"Multinomial NB", "accuracy":m2[0]*100, "features":m2[1], "label":label},\
            {"clf":"Linear SVM","accuracy":m3[0]*100, "features":m3[1], "label":label}]
    
    return pd.DataFrame(data)

In [ ]:
general_count = test_models(fit_count_model, demo_tweets, "english", "General Tweets \n(Count Vectorized)")
print ""
general_tfidf = test_models(fit_tfidf_model, demo_tweets, "english", "General Tweets \n(TF-IDF Vectorized)")
print ""
airline_count = test_models(fit_count_model, cf_train, air_travel_sws, "Airline Tweets \n(Count Vectorized)")
print ""
airline_tfidf = test_models(fit_tfidf_model, cf_train, air_travel_sws, "Airline Tweets \n(TF-IDF Vectorized)")
print ""

results = pd.concat([general_count, general_tfidf, airline_count, airline_tfidf], axis=0)
results

#### Create Accuracy Graph

In [ ]:
plt.figure(figsize=(20,10))

reds = sns.color_palette("Greens")[2:5]
blues = sns.color_palette("Blues")[2:5]

l1=plt.axhline(y=float(cf_test.sentiment.value_counts()[1]*100)/ cf_test.shape[0], color="black")
plt.axhline(y=lexicon_grid_search("text_blob_sentiment", cf_test, 0.05)[1]*100, color="#ff00bf")
plt.axhline(y=lexicon_grid_search("vader_sentiment", cf_test, 0.05)[1]*100, color="#8000ff")

sns.barplot(x="label", y="accuracy", hue="clf", palette=blues, data=results)

plt.xlabel("")
plt.ylabel("Accuracy (%)", fontsize=32)

plt.tick_params(axis="x", labelsize=28)
plt.tick_params(axis="y", labelsize=24)

plt.legend(loc="upper left", fontsize=28)

pylab.ylim([50,100])
plt.savefig("graphics/sentiment_analyzer_accuracy.png")

#### Look at Most Important Keywords

In [ ]:
print results.columns

In [ ]:
def print_top_unigrams(df, clf, label):
    print clf, label
    
    for each in df.ix[(df["clf"]==clf) & (df["label"]==label)]["features"].tolist()[0]:
        print each

In [ ]:
print_top_unigrams(results, "Bernoulli NB", "General Tweets \n(Count Vectorized)")
print_top_unigrams(results, "Bernoulli NB", "Airline Tweets \n(Count Vectorized)")

#### See if Upsampling General Tweets Makes any Difference

In [ ]:
def upsampler(df, n):
    return df.sample(n=n, replace=True)

